In [1]:
catalog.list()


[
    'companies',
    'reviews',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'model_input_table',
    'regressor',
    'scope3_data_3',
    'scope3_data_0',
    'preprocessed_scope3',
    'parameters',
    'params:feature_options',
    'params:feature_options.features',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.features'
]

In [2]:
import numpy as np
import pandas as pd
from typing import Dict

In [3]:
df = catalog.load("scope3_data_3")

                    INFO     Loading data from 'scope3_data_3' (ExcelDataSet)...                ]8;id=359623;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=145844;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py#492\492]8;;\

In [4]:
features = catalog.load("params:feature_options")

[09/19/23 13:05:21] INFO     Loading data from 'params:feature_options' (MemoryDataset)...      ]8;id=847381;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=140637;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py#492\492]8;;\

In [5]:
#pipelines

In [6]:
#%run_viz

In [7]:
def _remove_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to remove all rows with missing values in a pandas dataframe.

    Args:
        df (pd.DataFrame): Input pandas DataFrame

    Returns:
        pd.DataFrame: Output DataFrame with rows containing missing values removed.
    """

    df_cleaned = df.dropna()

    return df_cleaned

In [8]:
def preprocess_scope3(scope3_data: pd.DataFrame, parameters: Dict) -> pd.DataFrame:
    """Preprocesses the Scope 3 data.

    Args:
        scope3_data: Raw data.
        
    Returns:
        Preprocessed data, with missing values removed.
    """
    
    df = scope3_data[parameters["features"]]
    preprocessed_data = _remove_missing_values(df)
    return preprocessed_data

In [9]:
preprocessed_df = preprocess_scope3(df, features)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 preprocessed_df = preprocess_scope3(df, features)                                            │
│   2                                                                                              │
│                                                                                                  │
│ in preprocess_scope3:11                                                                          │
│                                                                                                  │
│    8 │   │   Preprocessed data, with missing values removed.                                     │
│    9 │   """                                                                                     │
│   10 │                                                                                           │
│ ❱ 11 │   df = scope3_data[parameters["features"]]                                                │
│   12 │   preprocessed_data = _remove_missing_values(df)                                          │
│   13 │   return preprocessed_data                                                                │
│   14                                                                                             │
│                                                                                                  │
│ /workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/pandas/core/frame.py:3767 in  │
│ __getitem__                                                                                      │
│                                                                                                  │
│    3764 │   │   else:                                                                            │
│    3765 │   │   │   if is_iterator(key):                                                         │
│    3766 │   │   │   │   key = list(key)                                                          │
│ ❱  3767 │   │   │   indexer = self.columns._get_indexer_strict(key, "columns")[1]                │
│    3768 │   │                                                                                    │
│    3769 │   │   # take() does not accept boolean indexers                                        │
│    3770 │   │   if getattr(indexer, "dtype", None) == bool:                                      │
│                                                                                                  │
│ /workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/pandas/core/indexes/base.py:5 │
│ 877 in _get_indexer_strict                                                                       │
│                                                                                                  │
│   5874 │   │   else:                                                                             │
│   5875 │   │   │   keyarr, indexer, new_indexer = self._reindex_non_unique(keyarr)               │
│   5876 │   │                                                                                     │
│ ❱ 5877 │   │   self._raise_if_missing(keyarr, indexer, axis_name)                                │
│   5878 │   │                                                                                     │
│   5879 │   │   keyarr = self.take(indexer)                                                       │
│   5880 │   │   if isinstance(key, Index):                                                        │
│                                                                                                  │
│ /workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/pandas/core/indexes/base.py:5 │
│ 941 in _raise_if_missing                                   

In [ ]:
def _remap_countries(df: pd.DataFrame) -> pd.DataFrame:
    # Define countries to keep based on a threshold of 50
    countries_to_keep = df['Country'].value_counts()[df['Country'].value_counts() > 50].index

    # Remap countries with counts less than 50 to 'Other'
    df['Country'] = df['Country'].apply(lambda x: x if x in countries_to_keep else 'Other')
    
    return df
    

In [ ]:
def _remap_industry(df: pd.DataFrame) -> pd.DataFrame:
    # Define industries to keep based on a threshold of 50
    industries_to_keep = df['Industry (Exiobase)'].value_counts()[df['Industry (Exiobase)'].value_counts() > 50].index

    # Remap industries with counts less than 50 to 'Other'
    df['Industry (Exiobase)'] = df['Industry (Exiobase)'].apply(lambda x: x if x in industries_to_keep else 'Other')
    
    return df

In [ ]:
def feature_engineering(scope3_data: pd.DataFrame) -> pd.DataFrame:
    df = _remap_countries(scope3_data)
    df_feature_engineered = _remap_industry(df)
    
    return df_feature_engineered
    

In [ ]:
df_feature_engineered = feature_engineering(preprocessed_df)

In [ ]:
df_feature_engineered

In [ ]:
features